# 🎯 MLFlow Experiment Tracking für die "Big 3" Algorithmen

> 🚀 **Motivation:**
 >
> Mit MLFlow und der QUA³CK A³-Schleife hebst du deine Machine-Learning-Projekte auf das nächste Level: Du vergleichst systematisch verschiedene Algorithmen, dokumentierst alle Experimente professionell und sammelst direkt Material für dein Portfolio und Bewerbungen.

**AMALEA 2025 – Woche 4, Erweiterte Integration**

> 🔄 **QUA³CK A³-Schleife:** Systematischer Vergleich von **Decision Trees**, **KNN** und **K-Means**
 >
 > 📊 **MLOps-Integration:** Professionelles Experiment-Tracking für Portfolio-Projekte
 >
 > 🚀 **Portfolio-Relevanz:** Zeigt MLOps-Kompetenzen in Bewerbungen

> 💡 **Warum lohnt sich das?**
- Du lernst, wie Profis Machine-Learning-Experimente reproduzierbar und nachvollziehbar machen.
- Du kannst deine Ergebnisse überzeugend präsentieren – ein echter Pluspunkt im Portfolio.
- Du sammelst praktische Erfahrung mit Tools, die in der Industrie Standard sind.

---

## 🎓 Integration mit dem Haupt-Notebook

Dieses Notebook erweitert `01_Bäume_Nachbarn_Clustering.ipynb` um professionelle MLOps-Praktiken aus dem QUA³CK-Handout.

> 📚 **Glossar-Tipp:** Unklare Begriffe? Schau ins [Glossar](../../01_Python_Grundlagen/02_Glossar_Alle_Begriffe_erklärt.ipynb) – dort findest du alle wichtigen Erklärungen!

### 📊 Was du hier lernst:
- ✅ **MLFlow Setup** für die Big 3 Algorithmen
- ✅ **Automatisiertes Hyperparameter-Tuning** mit Tracking
- ✅ **Model Comparison Dashboard**
- ✅ **Model Registry** für produktionsreife Modelle
- ✅ **Portfolio-Dokumentation** für GitHub

---

## 🔧 MLFlow Setup für AMALEA Big 3

> 💡 **Profi-Tipp**: MLFlow Tracking macht eure Experimente **reproduzierbar** und **portfolio-ready**!

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, silhouette_score, classification_report
from sklearn.datasets import load_iris, load_wine
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# MLFlow Experiment Setup
mlflow.set_experiment("AMALEA_2025_Big3_Algorithms")

print("🔧 MLFlow Setup für AMALEA Big 3 Algorithmen")
print("💡 Alle Experimente werden automatisch getrackt für euer Portfolio!")
print(f"📊 MLFlow UI: http://localhost:5001 (falls Docker läuft)")

## 📊 Dataset Selection & Preparation

> 🎯 **Portfolio-Tipp**: Nutzt verschiedene Datasets für eure Fallstudien!

In [ ]:
# Multiple Datasets für umfassende Tests
datasets = {
    'iris': load_iris(),
    'wine': load_wine()
}

print("📊 AMALEA Datasets für Big 3 Comparison:")
for name, dataset in datasets.items():
    print(f"  • {name.capitalize()}: {dataset.data.shape[0]} samples, {dataset.data.shape[1]} features, {len(dataset.target_names)} classes")

# Hauptdataset für Demo
iris = datasets['iris']
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\n🎯 Training Set: {X_train.shape[0]} samples")
print(f"🎯 Test Set: {X_test.shape[0]} samples")

## 🌲 Algorithm 1: Decision Trees (mit MLFlow)

> 🌳 **QUA³CK A³-Phase**: Systematische Hyperparameter-Optimierung mit Tracking

In [ ]:
def train_decision_tree_with_mlflow(X_train, X_test, y_train, y_test, dataset_name="iris"):
    """Decision Tree Training mit MLFlow Tracking"""
    
    # Hyperparameter Grid für A³-Schleife
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    best_accuracy = 0
    best_params = None
    
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            for min_samples_leaf in param_grid['min_samples_leaf']:
                
                with mlflow.start_run(run_name=f"DecisionTree_{dataset_name}"):
                    # Current parameters
                    params = {
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'random_state': 42
                    }
                    
                    # Model training
                    dt = DecisionTreeClassifier(**params)
                    dt.fit(X_train, y_train)
                    
                    # Predictions & Metrics
                    y_pred = dt.predict(X_test)
                    accuracy = accuracy_score(y_test, y_pred)
                    
                    # MLFlow Logging
                    mlflow.log_param("algorithm", "DecisionTree")
                    mlflow.log_param("dataset", dataset_name)
                    mlflow.log_params(params)
                    mlflow.log_metric("accuracy", accuracy)
                    mlflow.log_metric("train_size", len(X_train))
                    mlflow.log_metric("test_size", len(X_test))
                    
                    # Track best model
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = params
                        # Log best model
                        mlflow.sklearn.log_model(dt, "decision_tree_model")
    
    return best_accuracy, best_params

# Execute Decision Tree Experiments
print("🌲 Training Decision Trees mit MLFlow...")
dt_accuracy, dt_params = train_decision_tree_with_mlflow(X_train, X_test, y_train, y_test)
print(f"✅ Best Decision Tree: {dt_accuracy:.3f} accuracy")
print(f"📊 Best Params: {dt_params}")

## 👥 Algorithm 2: K-Nearest Neighbors (mit MLFlow)

> 🔍 **QUA³CK A³-Phase**: KNN Hyperparameter-Tuning mit Distance Metrics

In [ ]:
def train_knn_with_mlflow(X_train, X_test, y_train, y_test, dataset_name="iris"):
    """KNN Training mit MLFlow Tracking"""
    
    # Hyperparameter Grid
    k_values = [3, 5, 7, 9, 11]
    distance_metrics = ['euclidean', 'manhattan', 'minkowski']
    
    best_accuracy = 0
    best_params = None
    
    for k in k_values:
        for metric in distance_metrics:
            
            with mlflow.start_run(run_name=f"KNN_{dataset_name}"):
                # Current parameters
                params = {
                    'n_neighbors': k,
                    'metric': metric
                }
                
                # Model training
                knn = KNeighborsClassifier(**params)
                knn.fit(X_train, y_train)
                
                # Predictions & Metrics
                y_pred = knn.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                
                # MLFlow Logging
                mlflow.log_param("algorithm", "KNN")
                mlflow.log_param("dataset", dataset_name)
                mlflow.log_params(params)
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("k_value", k)
                
                # Track best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = params
                    mlflow.sklearn.log_model(knn, "knn_model")
    
    return best_accuracy, best_params

# Execute KNN Experiments
print("👥 Training K-Nearest Neighbors mit MLFlow...")
knn_accuracy, knn_params = train_knn_with_mlflow(X_train, X_test, y_train, y_test)
print(f"✅ Best KNN: {knn_accuracy:.3f} accuracy")
print(f"📊 Best Params: {knn_params}")

## 🎯 Algorithm 3: K-Means Clustering (mit MLFlow)

> 📊 **QUA³CK A³-Phase**: Unsupervised Learning Evaluation mit Silhouette Score

In [ ]:
def train_kmeans_with_mlflow(X_train, X_test, dataset_name="iris"):
    """K-Means Training mit MLFlow Tracking"""
    
    # Hyperparameter Grid
    k_values = [2, 3, 4, 5, 6]
    init_methods = ['k-means++', 'random']
    
    best_silhouette = -1
    best_params = None
    
    for k in k_values:
        for init_method in init_methods:
            
            with mlflow.start_run(run_name=f"KMeans_{dataset_name}"):
                # Current parameters
                params = {
                    'n_clusters': k,
                    'init': init_method,
                    'random_state': 42,
                    'n_init': 10
                }
                
                # Model training
                kmeans = KMeans(**params)
                kmeans.fit(X_train)
                
                # Predictions & Metrics
                train_labels = kmeans.predict(X_train)
                test_labels = kmeans.predict(X_test)
                
                # Silhouette Score (clustering quality)
                train_silhouette = silhouette_score(X_train, train_labels)
                test_silhouette = silhouette_score(X_test, test_labels)
                
                # MLFlow Logging
                mlflow.log_param("algorithm", "KMeans")
                mlflow.log_param("dataset", dataset_name)
                mlflow.log_params(params)
                mlflow.log_metric("train_silhouette", train_silhouette)
                mlflow.log_metric("test_silhouette", test_silhouette)
                mlflow.log_metric("inertia", kmeans.inertia_)
                
                # Track best model
                if test_silhouette > best_silhouette:
                    best_silhouette = test_silhouette
                    best_params = params
                    mlflow.sklearn.log_model(kmeans, "kmeans_model")
    
    return best_silhouette, best_params

# Execute K-Means Experiments
print("🎯 Training K-Means Clustering mit MLFlow...")
kmeans_score, kmeans_params = train_kmeans_with_mlflow(X_train, X_test)
print(f"✅ Best K-Means: {kmeans_score:.3f} silhouette score")
print(f"📊 Best Params: {kmeans_params}")

## 📊 QUA³CK Phase C: Model Comparison Dashboard

> 📈 **Portfolio-Highlight**: Professioneller Model Comparison für Bewerbungen!

In [ ]:
# Big 3 Results Summary
results_summary = {
    'Algorithm': ['Decision Tree', 'K-Nearest Neighbors', 'K-Means'],
    'Primary_Metric': [f'{dt_accuracy:.3f} (Accuracy)', f'{knn_accuracy:.3f} (Accuracy)', f'{kmeans_score:.3f} (Silhouette)'],
    'Best_Params': [str(dt_params), str(knn_params), str(kmeans_params)],
    'Use_Case': ['Classification', 'Classification', 'Clustering'],
    'Interpretability': ['High', 'Medium', 'Medium'],
    'Scalability': ['Medium', 'Low', 'High']
}

comparison_df = pd.DataFrame(results_summary)
print("📊 AMALEA Big 3 Algorithm Comparison:")
print("=" * 80)
print(comparison_df.to_string(index=False))

# Portfolio Summary
print("\n🚀 Portfolio Documentation:")
print("=" * 40)
print(f"✅ Experiments Tracked: {len(results_summary['Algorithm'])} algorithms")
print(f"✅ MLFlow Runs: Viewable at http://localhost:5001")
print(f"✅ Best Classification: {'Decision Tree' if dt_accuracy > knn_accuracy else 'KNN'} ({max(dt_accuracy, knn_accuracy):.3f})")
print(f"✅ Best Clustering: K-Means ({kmeans_score:.3f} silhouette)")
print("✅ Repository Ready: Models logged for production deployment")

## 🎯 QUA³CK Phase K: Knowledge Transfer & Model Registry

> 🏆 **Production Ready**: Register best models für Streamlit Apps!

In [ ]:
# Model Registry Preparation
print("🏆 QUA³CK Phase K: Knowledge Transfer")
print("=" * 50)

# Determine best models for each category
best_classifier = "Decision Tree" if dt_accuracy > knn_accuracy else "KNN"
best_classifier_score = max(dt_accuracy, knn_accuracy)

production_models = {
    "classification": {
        "algorithm": best_classifier,
        "accuracy": best_classifier_score,
        "use_case": "Iris Species Classification for Streamlit App"
    },
    "clustering": {
        "algorithm": "K-Means",
        "silhouette_score": kmeans_score,
        "use_case": "Customer Segmentation for Portfolio Projects"
    }
}

print("🚀 Production-Ready Models:")
for task, model_info in production_models.items():
    print(f"\n  📊 {task.capitalize()}:")
    for key, value in model_info.items():
        print(f"    • {key}: {value}")

# Next Steps for AMALEA Students
print("\n🎓 Next Steps für euer AMALEA Portfolio:")
print("  1. ✅ Notebook completed (Big 3 + MLFlow documented)")
print("  2. 🚀 Create Streamlit App mit bestem Classification Model")
print("  3. 📊 Add K-Means Clustering Visualization")
print("  4. ☁️ Deploy to Streamlit Cloud (public URL)")
print("  5. 📚 Add to GitHub Portfolio Repository")
print("  6. 🎯 Use as foundation für eure Fallstudien")

print("\n💡 Portfolio-Tipp: Diese MLFlow-Integration zeigt MLOps-Kompetenzen in Bewerbungen!")

## 📈 Visualization Dashboard

> 📊 **Portfolio-Feature**: Professional Model Performance Visualization

In [ ]:
# Create Portfolio-Ready Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('AMALEA 2025: Big 3 Algorithms Performance Dashboard', fontsize=16, fontweight='bold')

# 1. Algorithm Comparison
algorithms = ['Decision Tree', 'KNN', 'K-Means']
scores = [dt_accuracy, knn_accuracy, kmeans_score]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

bars = axes[0,0].bar(algorithms[:2], scores[:2], color=colors[:2])
axes[0,0].set_title('Classification Accuracy Comparison')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].set_ylim(0, 1)
for bar, score in zip(bars, scores[:2]):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{score:.3f}', ha='center', fontweight='bold')

# 2. K-Means Silhouette
axes[0,1].bar(['K-Means'], [kmeans_score], color=colors[2])
axes[0,1].set_title('Clustering Quality (Silhouette Score)')
axes[0,1].set_ylabel('Silhouette Score')
axes[0,1].set_ylim(0, 1)
axes[0,1].text(0, kmeans_score + 0.01, f'{kmeans_score:.3f}', ha='center', fontweight='bold')

# 3. Feature Importance (simulated for Demo)
feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
importance = [0.1, 0.05, 0.6, 0.25]  # Simulated importance
axes[1,0].barh(feature_names, importance, color='skyblue')
axes[1,0].set_title('Feature Importance (Decision Tree)')
axes[1,0].set_xlabel('Importance')

# 4. Performance Summary
axes[1,1].axis('off')
summary_text = f"""
🏆 AMALEA 2025 Results Summary

📊 Best Classifier: {best_classifier}
   Accuracy: {best_classifier_score:.1%}

🎯 Clustering Quality: {kmeans_score:.3f}
   Algorithm: K-Means

🔬 Total Experiments: 50+ MLFlow Runs
🚀 Production Ready: ✅
📚 Portfolio Ready: ✅
"""
axes[1,1].text(0.1, 0.5, summary_text, fontsize=12, verticalalignment='center',
               bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))

plt.tight_layout()
plt.show()

print("📊 Portfolio-Dashboard erstellt!")
print("💡 Speichere diese Visualisierung für deine Bewerbungsunterlagen!")

## 🎯 Zusammenfassung: QUA³CK + MLOps für Portfolio

### ✅ Was ihr erreicht habt:

1. **QUA³CK A³-Schleife implementiert** mit systematischem Algorithm Comparison
2. **MLFlow Experiment Tracking** für professionelle ML-Workflows
3. **Big 3 Algorithmen mastered**: Decision Trees, KNN, K-Means
4. **Production-Ready Models** für Streamlit App Integration
5. **Portfolio Documentation** für GitHub und Bewerbungen

### 🚀 Next Steps für AMALEA:

- **Woche 5**: Neural Networks mit derselben MLFlow-Integration
- **Woche 6**: Computer Vision & NLP mit Model Registry
- **Woche 7**: Full MLOps Pipeline für Production Deployment
- **Fallstudien**: Nutzt diese MLFlow-Kompetenzen für eure Assessment-Projekte

### 💼 Portfolio-Highlights:

✅ **Reproducible Experiments** (MLFlow Tracking)  
✅ **Algorithm Comparison** (Data-Driven Model Selection)  
✅ **Production Readiness** (Model Registry Integration)  
✅ **Professional Documentation** (GitHub + MLFlow UI)  
✅ **Industry Standards** (MLOps Best Practices)  

🎯 **Diese MLOps-Integration zeigt euren zukünftigen Arbeitgebern, dass ihr nicht nur ML-Algorithmen versteht, sondern auch professionelle Engineering-Workflows beherrscht!**

---

*AMALEA 2025 - QUA³CK + MLOps = Portfolio-Ready Data Scientists* ✨